<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/year_built_classifier.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/year_built_classifier.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [1]:
!pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

  Cloning https://github.com/bacetiner/BrailsPlusPlus to /tmp/pip-req-build-ft1_2sd3
  Running command git clone --filter=blob:none --quiet https://github.com/bacetiner/BrailsPlusPlus /tmp/pip-req-build-ft1_2sd3
  Resolved https://github.com/bacetiner/BrailsPlusPlus to commit 2e67ed4d5e8c735f7eab0178057c02f5389e0217
  Preparing metadata (setup.py) ... done
  Created wheel for brails: filename=brails-4.0-py3-none-any.whl size=2024144 sha256=b1af3309b92b22a0bc62a4ce8775692a9e8ec90e0194e6557ffd13e108a9a100
  Stored in directory: /tmp/pip-ephem-wheel-cache-158l70x1/wheels/65/0b/aa/a9113e0557b48ca6dfe52e54ff2df5af4aa9945a3736b53797
Successfully built brails


## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the `YearBuiltClassifier` demonstrated in this example.

The `Importer` class provides the main methods to load these modules. To gather the images needed for attribute prediction, users can use the ImageSet class or BRAILS++ scrapers, which facilitate automated asset geometry and image retrieval. This example illustrates both methods for sourcing imagery for use with the `YearBuiltClassifier` module.

In [2]:
from brails import Importer
from brails.types.image_set import ImageSet

## **Define Google API Key**
You need a Google API Key with Street View Static API enabled to download the Google Street View imagery required for this example. Please follow this [link](https://support.google.com/googleapi/answer/6158862?hl=en) for instructions on setting up a Google API key.

In [3]:
API_KEY = 'YOUR-API-KEY-HERE'

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `YearBuiltClassifier` class identifies the construction era of a building from street-level imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.
1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, street-level imagery of buildings in Lake Charles, LA, are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Lake Charles, Louisiana`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the Microsoft footprint data through the `MS_FootprintScraper`.</strong> The only information required to initialize `MS_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `MS_FootprintScraper` is an `AssetInventory` saved in `lake_charles_fp`.

3. <strong>Get a randomly selected `100` building subset of the footprint inventory obtained using `lake_charles_fp`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `lake_charles_fp_subset`. For this example, the random seed is set to `15` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleStreetview` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `lake_charles_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/street/`.

In [4]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName",
               "data": "Lake Charles, Louisiana"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

ms_fp_class = importer.get_class("MS_FootprintScraper")
ms_fp_data = {"length": "ft"}
footprint_scraper = ms_fp_class(ms_fp_data)
lake_charles_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 20 buildings using a random seed value of 40:
lake_charles_fp_subset = lake_charles_fp.get_random_sample(100, 40)

# Get aerial imagery for the selected subset using GoogleSatellite:
google_street_class = importer.get_class("GoogleStreetview")
google_street = google_street_class({'apiKey': API_KEY})
images_street = google_street.get_images(
    lake_charles_fp_subset, "tmp/street/")


Searching for Lake Charles, Louisiana...
Found Lake Charles, Calcasieu Parish, Louisiana, United States


Processing quadkeys: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.67s/it]



Found a total of 34913 building footprints in Lake Charles


INFO:brails.scrapers.google_streetview.google_streetview:
Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/image_processor/tmp/street

Obtaining street-level imagery:   1%|█▏                                                                                                                       | 1/100 [00:00<00:10,  9.84it/s]INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 30.1640, -93.2614
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 30.1645, -93.2106
Obtaining street-level imagery:  30%|████████████████████████████████████                                                                                    | 30/100 [00:03<00:08,  8.51it/s]INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 30.1851, -93.1746
INFO:brails.scrapers.google_streetview.google_streetview:No 

## **Predict Building Construction Eras Using the YearBuiltClassifier Module**
`YearBuiltClassifier` includes a pre-trained ResNet-50 model that has been trained on a custom dataset of 10,000 images. This model is capable of predicting six eras of construction 1) Pre-1970, 2) 1970 - 1979, 3) 1980 - 1989, 4) 1990 - 1999, 5) 2000 - 2009, and 6) Post-2010, denoted with `1960`, `1975`, `1985`, `1995`, `2005`, and `2015` respectively.

In [5]:
importer = Importer()
consyear_classifier_class = importer.get_class('YearBuiltClassifier')
consyear_classifier = consyear_classifier_class()
predictions = consyear_classifier.predict(images_street)

Default model checkpoint found at tmp/models.
Performing construction era classifications...


/home/bacetiner/anaconda3/envs/brails_plus/lib/python3.11/site-packages/brails/processors/year_built_classifier/year_built_classifier.py:161: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimen

## **Show Prediction Results**

In [6]:
print(predictions)

{20375: 1985, 20693: 1985, 21302: 1960, 30055: 1960, 6394: 1960, 30265: 1975, 18509: 1985, 13958: 1960, 28404: 1960, 18679: 1975, 13522: 1960, 15759: 1985, 9490: 1960, 31406: 1960, 8511: 1960, 29733: 1985, 28201: 1985, 25223: 1975, 22832: 1975, 34339: 1960, 16075: 1985, 3216: 1975, 3646: 1960, 34104: 1960, 10656: 2005, 16563: 1960, 6015: 1960, 3484: 1960, 10353: 1960, 11324: 1985, 7445: 1975, 6506: 1960, 33838: 1960, 15481: 2005, 7070: 1960, 12232: 1985, 9743: 1985, 30234: 1960, 13792: 1960, 10698: 1960, 14587: 1975, 33623: 1960, 33555: 1960, 16414: 1960, 34040: 1960, 22015: 1960, 13237: 1995, 1876: 1960, 14881: 1960, 19872: 1960, 16329: 1960, 29699: 1960, 15787: 1995, 33172: 1960, 13433: 1960, 30190: 1960, 2433: 1960, 3886: 1960, 11193: 1960, 32243: 1960, 12525: 1960, 11570: 1960, 18161: 1960, 15854: 1960, 12245: 1960, 20233: 1985, 28855: 1960, 10459: 1960, 20299: 1960, 27471: 1960, 19096: 1960, 20526: 1975, 10080: 1960, 34300: 1960, 20006: 1960, 7357: 1960, 30893: 1985, 27401: 1960, 